In [1]:
cd ..

/Users/muhammadfarooq/Codebase/wood-log-detection/Pith2Bark


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scikitplot as skplt

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Dropout

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from src.cv_winner import winner_model_rings
from src.damage import merge_df
from src.damage import read_damage_min_max
from src.damage import read_damage_mean

# Damage Dataset

In [3]:
rmse_url = "cv_output/ringsranking_rmse.csv"
rings_url = "cv_output/ringsranking_rings.csv"
df_ringsranking = winner_model_rings(rmse_url, rings_url)

Polar Pith Canny(blur otsu) 12.829 (subseq)


In [4]:
rmse_url = "cv_output/pine_rmse.csv"
rings_url = "cv_output/pine_rings.csv"
df_pine = winner_model_rings(rmse_url, rings_url)

Polar Pith Canny(blur triangle) 19.888 (peaks)


In [5]:
rmse_url = "cv_output/fur_rmse.csv"
rings_url = "cv_output/fur_rings.csv"
df_fur = winner_model_rings(rmse_url, rings_url)

Polar Pith Canny(blur triangle) 13.429 (peaks)


In [6]:
rmse_url = "cv_output/tracy_rmse.csv"
rings_url = "cv_output/tracy_rings.csv"
df_tracy = winner_model_rings(rmse_url, rings_url)

Polar Canny(triangle) 39.684 (peaks)


In [7]:
rmse_url = "cv_output/data_rmse.csv"
rings_url = "cv_output/data_rings.csv"
df_data = winner_model_rings(rmse_url, rings_url)

Canny(triangle) 18.79 (subseq)


In [8]:
#damage
damage_url = "damage_dataset/damage.csv"

#merge
df_merge = merge_df(df_ringsranking, df_pine, df_fur, df_tracy, df_data)

df_damage_min_max = read_damage_min_max(damage_url, df_merge)

df_damage_mean = read_damage_mean(damage_url, df_merge)

In [9]:
#STOP

# Config

In [10]:
training_percent = 0.9

epochs, batch_size = 50, 32
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# mode = max for monitro val_accuracy and val_prc
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=10,
    restore_best_weights=True)

METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'), 
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

#columns to drop
columns = ['image', 'algo']

In [11]:
def heat_map_normalized(y_true, y_pred):
    
    class_names = ['Not Damage', 'Damaged']
    
    skplt.metrics.plot_confusion_matrix(y_true, y_pred,
                                        figsize=(4,3),
                                        normalize=True)
    
    # Customize axis tick labels
    plt.xticks([0,1], class_names)
    plt.yticks([0,1], class_names)
    
    plt.yticks(rotation=90)
    
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

In [12]:
def heat_map(_cm):
    # plot confusion matrix as heatmap
    labels = ['Not Damage', 'Damaged']
    
    # Set up the matplotlib figure
    plt.figure(figsize=(4, 3))
    
    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    
    sns.heatmap(_cm, annot=True, cmap=cmap, xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

In [13]:
units = [64]
activations = ['relu', 'tanh', 'sigmoid']
kernel_size = [3, 5]
learning_rate = [0.001, 0.0001, 0.00001]
optimizer = ['Adam', 'RMSprop', 'Nadam']
dropout = [0.1, 0.2, 0.3]

# Model

In [14]:
class Baseline():
    def __init__(self, _input_dim, unit, activation, kernel_size, learning_rate, optimizer, dropout, metrics=METRICS):
        # Define the model architecture
        model = Sequential()
        
        # hidden layer
        model.add(Dense(unit, activation=activation, input_dim=_input_dim))
        model.add(Dropout(dropout))
        
        # hidden layer
        model.add(Dense(unit, activation=activation))
        model.add(Dropout(dropout))
        
        # hidden layer
        model.add(Dense(unit, activation=activation))
        model.add(Dropout(dropout))
        
        # hidden layer
        model.add(Dense(unit, activation=activation))
        model.add(Dropout(dropout))
        
        # output layer
        model.add(Dense(1, activation='sigmoid'))

        if optimizer == 'Adam':
            ops = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        elif optimizer == 'RMSprop':
            ops = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        elif optimizer == 'Nadam':
            ops = tf.keras.optimizers.Nadam(learning_rate=learning_rate)
            
        # Compile the model
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        self.model = model
        
    def train(self, X_train, y_train, X_val, y_val, epochs, batch_size):
        history = self.model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[early_stopping], verbose=2)
        return history

    def evaluate(self, X_test, y_test):
        results = self.model.evaluate(X_test, y_test, verbose=0)
        return results

    def predict(self, X): #X_train X_test
        predictions = self.model.predict(X)
        return predictions
    
    def summary(self):
        self.model.summary()
        
    def metrics_names(self):
        return self.model.metrics_names

# Model MinMax

In [15]:
df = df_damage_min_max.drop(columns=columns)

X = df.drop(columns=['Damage'])
y = df['Damage']

# split the data into a holdout set and the rest for K-Fold cross-validation
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, train_size=training_percent, random_state=42)

In [16]:
# Initialize StandardScaler
scaler = StandardScaler()

# Fit scaler on training data
scaler.fit(X_train)

# Transform both training and test data
X_train_scaled = scaler.transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

In [17]:
best_acc = 0.0
baseline_values = { 'units': 0, 'activation': '', 'kernel_size': 0, 'learning_rate' : 0.0, 'optimizer': '', 'dropout': 0.0 }

In [18]:
input_dim = X_train.shape[1]

In [19]:
for u in units:
    for a in activations:
        for k in kernel_size:
            for l in learning_rate:
                for o in optimizer:
                    for d in dropout:
                        print('\nUnits:', u, 'Activation:', a, 'Kernel size:', k, 'Learning rate:', l, 'Optimizer:', o, 'Dropout:', d)
                        for train_index, val_index in kf.split(X_train_scaled):
                            # split the dataset into training and validation sets for this fold
                            X_train_kf, X_val_kf = X_train_scaled[train_index], X_train_scaled[val_index]
                            y_train_kf, y_val_kf = y_train.iloc[train_index], y_train.iloc[val_index]

                            model = Baseline(input_dim, u, a, k, l, o, d)

                            history = model.train(X_train_kf, y_train_kf, X_val_kf, y_val_kf, epochs, batch_size)
                            if (history.history['val_accuracy'][-1] > best_acc):
                                best_acc = history.history['val_accuracy'][-1]
                                baseline_values = { 'units': u, 'activation': a, 'kernel_size': k, 'learning_rate' : l, 'optimizer': o, 'dropout': d }


Units: 64 Activation: relu Kernel size: 3 Learning rate: 0.001 Optimizer: Adam Dropout: 0.1
Epoch 1/50
3/3 - 1s - loss: 0.6525 - accuracy: 0.3750 - val_loss: 0.5699 - val_accuracy: 0.8667 - 1s/epoch - 465ms/step
Epoch 2/50
3/3 - 0s - loss: 0.5743 - accuracy: 0.7045 - val_loss: 0.5118 - val_accuracy: 0.9333 - 34ms/epoch - 11ms/step
Epoch 3/50
3/3 - 0s - loss: 0.5165 - accuracy: 0.8409 - val_loss: 0.4687 - val_accuracy: 0.9333 - 34ms/epoch - 11ms/step
Epoch 4/50
3/3 - 0s - loss: 0.4713 - accuracy: 0.8523 - val_loss: 0.4308 - val_accuracy: 0.9333 - 33ms/epoch - 11ms/step
Epoch 5/50
3/3 - 0s - loss: 0.4166 - accuracy: 0.8523 - val_loss: 0.3972 - val_accuracy: 0.9333 - 33ms/epoch - 11ms/step
Epoch 6/50
3/3 - 0s - loss: 0.3638 - accuracy: 0.8864 - val_loss: 0.3671 - val_accuracy: 0.9333 - 34ms/epoch - 11ms/step
Epoch 7/50
3/3 - 0s - loss: 0.3251 - accuracy: 0.8636 - val_loss: 0.3410 - val_accuracy: 0.9333 - 35ms/epoch - 12ms/step
Epoch 8/50
3/3 - 0s - loss: 0.2832 - accuracy: 0.8977 - val_l

Epoch 31/50
3/3 - 0s - loss: 0.1373 - accuracy: 0.9663 - val_loss: 0.1975 - val_accuracy: 0.9318 - 42ms/epoch - 14ms/step
Epoch 32/50
3/3 - 0s - loss: 0.1350 - accuracy: 0.9663 - val_loss: 0.2020 - val_accuracy: 0.9318 - 35ms/epoch - 12ms/step
Epoch 33/50
3/3 - 0s - loss: 0.1084 - accuracy: 0.9663 - val_loss: 0.2049 - val_accuracy: 0.9318 - 28ms/epoch - 9ms/step
Epoch 34/50
3/3 - 0s - loss: 0.1196 - accuracy: 0.9663 - val_loss: 0.2166 - val_accuracy: 0.9318 - 32ms/epoch - 11ms/step
Epoch 35/50
3/3 - 0s - loss: 0.0872 - accuracy: 0.9663 - val_loss: 0.2330 - val_accuracy: 0.9091 - 37ms/epoch - 12ms/step

Units: 64 Activation: relu Kernel size: 3 Learning rate: 0.001 Optimizer: Adam Dropout: 0.2
Epoch 1/50
3/3 - 1s - loss: 0.6532 - accuracy: 0.7273 - val_loss: 0.6066 - val_accuracy: 0.8444 - 965ms/epoch - 322ms/step
Epoch 2/50
3/3 - 0s - loss: 0.5850 - accuracy: 0.8523 - val_loss: 0.5522 - val_accuracy: 0.8889 - 31ms/epoch - 10ms/step
Epoch 3/50
3/3 - 0s - loss: 0.5223 - accuracy: 0.8636 

Epoch 25/50
3/3 - 0s - loss: 0.1695 - accuracy: 0.9551 - val_loss: 0.1402 - val_accuracy: 0.9545 - 33ms/epoch - 11ms/step
Epoch 26/50
3/3 - 0s - loss: 0.1826 - accuracy: 0.9213 - val_loss: 0.1303 - val_accuracy: 0.9545 - 34ms/epoch - 11ms/step
Epoch 27/50
3/3 - 0s - loss: 0.1803 - accuracy: 0.9326 - val_loss: 0.1201 - val_accuracy: 0.9545 - 34ms/epoch - 11ms/step
Epoch 28/50
3/3 - 0s - loss: 0.1736 - accuracy: 0.9438 - val_loss: 0.1162 - val_accuracy: 0.9545 - 37ms/epoch - 12ms/step
Epoch 29/50
3/3 - 0s - loss: 0.1558 - accuracy: 0.9438 - val_loss: 0.1168 - val_accuracy: 0.9545 - 32ms/epoch - 11ms/step
Epoch 30/50
3/3 - 0s - loss: 0.1262 - accuracy: 0.9551 - val_loss: 0.1136 - val_accuracy: 0.9545 - 53ms/epoch - 18ms/step
Epoch 31/50
3/3 - 0s - loss: 0.1397 - accuracy: 0.9775 - val_loss: 0.1105 - val_accuracy: 0.9545 - 66ms/epoch - 22ms/step
Epoch 32/50
3/3 - 0s - loss: 0.1045 - accuracy: 0.9888 - val_loss: 0.1057 - val_accuracy: 0.9545 - 53ms/epoch - 18ms/step
Epoch 33/50
3/3 - 0s - l

Epoch 19/50
3/3 - 0s - loss: 0.2352 - accuracy: 0.9213 - val_loss: 0.4110 - val_accuracy: 0.8636 - 35ms/epoch - 12ms/step
Epoch 1/50
3/3 - 1s - loss: 0.7115 - accuracy: 0.4157 - val_loss: 0.6847 - val_accuracy: 0.2727 - 1s/epoch - 396ms/step
Epoch 2/50
3/3 - 0s - loss: 0.6354 - accuracy: 0.4719 - val_loss: 0.6167 - val_accuracy: 0.6364 - 32ms/epoch - 11ms/step
Epoch 3/50
3/3 - 0s - loss: 0.6018 - accuracy: 0.5843 - val_loss: 0.5665 - val_accuracy: 0.9091 - 32ms/epoch - 11ms/step
Epoch 4/50
3/3 - 0s - loss: 0.5979 - accuracy: 0.5730 - val_loss: 0.5286 - val_accuracy: 0.9318 - 33ms/epoch - 11ms/step
Epoch 5/50
3/3 - 0s - loss: 0.5468 - accuracy: 0.7640 - val_loss: 0.4938 - val_accuracy: 0.9318 - 41ms/epoch - 14ms/step
Epoch 6/50
3/3 - 0s - loss: 0.5338 - accuracy: 0.8427 - val_loss: 0.4612 - val_accuracy: 0.9318 - 31ms/epoch - 10ms/step
Epoch 7/50
3/3 - 0s - loss: 0.4965 - accuracy: 0.8876 - val_loss: 0.4290 - val_accuracy: 0.9318 - 32ms/epoch - 11ms/step
Epoch 8/50
3/3 - 0s - loss: 0.49

Epoch 17/50
3/3 - 0s - loss: 0.1585 - accuracy: 0.9545 - val_loss: 0.3465 - val_accuracy: 0.9111 - 32ms/epoch - 11ms/step
Epoch 18/50
3/3 - 0s - loss: 0.1412 - accuracy: 0.9659 - val_loss: 0.3630 - val_accuracy: 0.9111 - 33ms/epoch - 11ms/step
Epoch 1/50
3/3 - 1s - loss: 0.6138 - accuracy: 0.6517 - val_loss: 0.4827 - val_accuracy: 0.8864 - 981ms/epoch - 327ms/step
Epoch 2/50
3/3 - 0s - loss: 0.4111 - accuracy: 0.8989 - val_loss: 0.3804 - val_accuracy: 0.8864 - 35ms/epoch - 12ms/step
Epoch 3/50
3/3 - 0s - loss: 0.3027 - accuracy: 0.9213 - val_loss: 0.3463 - val_accuracy: 0.8864 - 38ms/epoch - 13ms/step
Epoch 4/50
3/3 - 0s - loss: 0.2695 - accuracy: 0.9101 - val_loss: 0.3332 - val_accuracy: 0.8864 - 37ms/epoch - 12ms/step
Epoch 5/50
3/3 - 0s - loss: 0.2332 - accuracy: 0.9101 - val_loss: 0.3312 - val_accuracy: 0.8864 - 35ms/epoch - 12ms/step
Epoch 6/50
3/3 - 0s - loss: 0.2231 - accuracy: 0.9101 - val_loss: 0.3368 - val_accuracy: 0.8864 - 33ms/epoch - 11ms/step
Epoch 7/50
3/3 - 0s - loss: 

...

Epoch 50/50
3/3 - 0s - loss: 0.4284 - accuracy: 0.8315 - val_loss: 0.2211 - val_accuracy: 0.9318 - 33ms/epoch - 11ms/step

Units: 64 Activation: sigmoid Kernel size: 5 Learning rate: 1e-05 Optimizer: Nadam Dropout: 0.1
Epoch 1/50
3/3 - 1s - loss: 0.6852 - accuracy: 0.5795 - val_loss: 0.6797 - val_accuracy: 0.5778 - 1s/epoch - 442ms/step
Epoch 2/50
3/3 - 0s - loss: 0.6616 - accuracy: 0.6591 - val_loss: 0.6808 - val_accuracy: 0.5778 - 32ms/epoch - 11ms/step
Epoch 3/50
3/3 - 0s - loss: 0.6629 - accuracy: 0.6364 - val_loss: 0.6859 - val_accuracy: 0.5778 - 32ms/epoch - 11ms/step
Epoch 4/50
3/3 - 0s - loss: 0.6329 - accuracy: 0.6705 - val_loss: 0.6939 - val_accuracy: 0.5778 - 30ms/epoch - 10ms/step
Epoch 5/50
3/3 - 0s - loss: 0.6429 - accuracy: 0.6705 - val_loss: 0.6973 - val_accuracy: 0.5778 - 31ms/epoch - 10ms/step
Epoch 6/50
3/3 - 0s - loss: 0.6443 - accuracy: 0.6591 - val_loss: 0.7001 - val_accuracy: 0.5778 - 30ms/epoch - 10ms/step
Epoch 7/50
3/3 - 0s - loss: 0.6488 - accuracy: 0.6705 - 

Epoch 7/50
3/3 - 0s - loss: 0.6576 - accuracy: 0.5955 - val_loss: 0.6095 - val_accuracy: 0.7273 - 32ms/epoch - 11ms/step
Epoch 8/50
3/3 - 0s - loss: 0.6747 - accuracy: 0.5618 - val_loss: 0.6101 - val_accuracy: 0.7273 - 30ms/epoch - 10ms/step
Epoch 9/50
3/3 - 0s - loss: 0.6948 - accuracy: 0.5618 - val_loss: 0.6075 - val_accuracy: 0.7273 - 32ms/epoch - 11ms/step
Epoch 10/50
3/3 - 0s - loss: 0.6870 - accuracy: 0.5730 - val_loss: 0.6050 - val_accuracy: 0.7273 - 31ms/epoch - 10ms/step
Epoch 11/50
3/3 - 0s - loss: 0.6627 - accuracy: 0.6067 - val_loss: 0.6027 - val_accuracy: 0.7273 - 31ms/epoch - 10ms/step
Epoch 12/50
3/3 - 0s - loss: 0.6687 - accuracy: 0.6180 - val_loss: 0.5959 - val_accuracy: 0.7273 - 32ms/epoch - 11ms/step
Epoch 13/50
3/3 - 0s - loss: 0.6694 - accuracy: 0.5955 - val_loss: 0.5918 - val_accuracy: 0.7273 - 31ms/epoch - 10ms/step
Epoch 14/50
3/3 - 0s - loss: 0.6458 - accuracy: 0.6180 - val_loss: 0.5840 - val_accuracy: 0.7273 - 32ms/epoch - 11ms/step
Epoch 15/50
3/3 - 0s - loss

Epoch 12/50
3/3 - 0s - loss: 0.6465 - accuracy: 0.6517 - val_loss: 0.6503 - val_accuracy: 0.6136 - 32ms/epoch - 11ms/step
Epoch 13/50
3/3 - 0s - loss: 0.6264 - accuracy: 0.6404 - val_loss: 0.6455 - val_accuracy: 0.6136 - 31ms/epoch - 10ms/step
Epoch 14/50
3/3 - 0s - loss: 0.6132 - accuracy: 0.6517 - val_loss: 0.6403 - val_accuracy: 0.6136 - 31ms/epoch - 10ms/step
Epoch 15/50
3/3 - 0s - loss: 0.6116 - accuracy: 0.6517 - val_loss: 0.6339 - val_accuracy: 0.6136 - 31ms/epoch - 10ms/step
Epoch 16/50
3/3 - 0s - loss: 0.6332 - accuracy: 0.6629 - val_loss: 0.6261 - val_accuracy: 0.6136 - 32ms/epoch - 11ms/step
Epoch 17/50
3/3 - 0s - loss: 0.6118 - accuracy: 0.6629 - val_loss: 0.6184 - val_accuracy: 0.6136 - 32ms/epoch - 11ms/step
Epoch 18/50
3/3 - 0s - loss: 0.6140 - accuracy: 0.6517 - val_loss: 0.6088 - val_accuracy: 0.6136 - 31ms/epoch - 10ms/step
Epoch 19/50
3/3 - 0s - loss: 0.5859 - accuracy: 0.6404 - val_loss: 0.5972 - val_accuracy: 0.6136 - 31ms/epoch - 10ms/step
Epoch 20/50
3/3 - 0s - l

Epoch 7/50
3/3 - 0s - loss: 0.6198 - accuracy: 0.6404 - val_loss: 0.6709 - val_accuracy: 0.6136 - 31ms/epoch - 10ms/step
Epoch 8/50
3/3 - 0s - loss: 0.6605 - accuracy: 0.6404 - val_loss: 0.6723 - val_accuracy: 0.6136 - 32ms/epoch - 11ms/step
Epoch 9/50
3/3 - 0s - loss: 0.6900 - accuracy: 0.6180 - val_loss: 0.6714 - val_accuracy: 0.6136 - 32ms/epoch - 11ms/step
Epoch 10/50
3/3 - 0s - loss: 0.6574 - accuracy: 0.6517 - val_loss: 0.6688 - val_accuracy: 0.6136 - 30ms/epoch - 10ms/step
Epoch 11/50
3/3 - 0s - loss: 0.6392 - accuracy: 0.6517 - val_loss: 0.6654 - val_accuracy: 0.6136 - 32ms/epoch - 11ms/step
Epoch 12/50
3/3 - 0s - loss: 0.6474 - accuracy: 0.6067 - val_loss: 0.6616 - val_accuracy: 0.6136 - 33ms/epoch - 11ms/step
Epoch 13/50
3/3 - 0s - loss: 0.6923 - accuracy: 0.5955 - val_loss: 0.6578 - val_accuracy: 0.6136 - 31ms/epoch - 10ms/step
Epoch 14/50
3/3 - 0s - loss: 0.6454 - accuracy: 0.6292 - val_loss: 0.6550 - val_accuracy: 0.6136 - 32ms/epoch - 11ms/step
Epoch 15/50
3/3 - 0s - loss

Epoch 25/50
3/3 - 0s - loss: 0.6450 - accuracy: 0.6517 - val_loss: 0.5697 - val_accuracy: 0.7273 - 33ms/epoch - 11ms/step
Epoch 26/50
3/3 - 0s - loss: 0.6276 - accuracy: 0.6180 - val_loss: 0.5614 - val_accuracy: 0.7273 - 32ms/epoch - 11ms/step
Epoch 27/50
3/3 - 0s - loss: 0.6509 - accuracy: 0.6180 - val_loss: 0.5495 - val_accuracy: 0.7273 - 31ms/epoch - 10ms/step
Epoch 28/50
3/3 - 0s - loss: 0.6164 - accuracy: 0.6517 - val_loss: 0.5351 - val_accuracy: 0.7273 - 30ms/epoch - 10ms/step
Epoch 29/50
3/3 - 0s - loss: 0.6129 - accuracy: 0.6180 - val_loss: 0.5219 - val_accuracy: 0.8636 - 31ms/epoch - 10ms/step
Epoch 30/50
3/3 - 0s - loss: 0.5941 - accuracy: 0.6517 - val_loss: 0.5074 - val_accuracy: 0.9091 - 32ms/epoch - 11ms/step
Epoch 31/50
3/3 - 0s - loss: 0.6168 - accuracy: 0.6404 - val_loss: 0.4908 - val_accuracy: 0.9318 - 33ms/epoch - 11ms/step
Epoch 32/50
3/3 - 0s - loss: 0.5629 - accuracy: 0.7416 - val_loss: 0.4727 - val_accuracy: 0.9773 - 32ms/epoch - 11ms/step
Epoch 33/50
3/3 - 0s - l

In [20]:
print(best_acc)
print(baseline_values)

1.0
{'units': 64, 'activation': 'tanh', 'kernel_size': 5, 'learning_rate': 1e-05, 'optimizer': 'RMSprop', 'dropout': 0.1}


In [21]:
#STOP